In [26]:
import numpy
from sklearn.metrics import accuracy_score
import tensorflow as tf
import matplotlib.pyplot as plt

from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.shapelets import LearningShapelets, \
    grabocka_params_to_shapelet_size_dict
from tslearn.utils import ts_size
import numpy
from sklearn.metrics import accuracy_score

from tslearn.generators import random_walk_blobs
from tslearn.preprocessing import TimeSeriesScalerMinMax, \
    TimeSeriesScalerMeanVariance
from tslearn.neighbors import KNeighborsTimeSeriesClassifier, \
    KNeighborsTimeSeries
import pandas as pd
import numpy as np
import random
from tslearn.datasets import UCR_UEA_datasets
from tslearn.preprocessing import TimeSeriesResampler

In [27]:
df = pd.read_pickle("Train.pkl")
randomlist1 = random.sample(range(0, 260), 78)

df_test1 = df.loc[df["Batch"].isin(randomlist1)]
df_train1 = df.loc[~df["Batch"].isin(randomlist1)]
df_steps1 = df_test1.loc[df_test1["CuStepNo ValueY"] != df_test1["Next_Step"]]
df_steps2 = df_train1.loc[df_train1["CuStepNo ValueY"] != df_train1["Next_Step"]]


In [64]:
y_train1 = df_steps2.loc[df_steps2["CuStepNo ValueY"] == 1,"DeviationID ValueY"].astype(int)
y_test1 = df_steps1.loc[df_steps1["CuStepNo ValueY"] == 1,"DeviationID ValueY"].astype(int)
X_train1 = df_train1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_test1 = df_test1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)

In [35]:
X_train_step1 = []
for i in X_train1.Batch.unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == 1)].drop(["Batch"],axis=1))
        X_train_step1.append(num)

In [66]:
X_test_step1 = []
for i in X_test1.Batch.unique():
        num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)&(X_test1["CuStepNo ValueY"] == 1)].drop(["Batch"],axis=1))
        X_test_step1.append(num)

In [45]:
length = []
for i in X_train_step1:
    length.append(len(i))

In [57]:
int(np.median(length))

248

In [67]:
X_train_s1 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_train_step1)
X_test_s1 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_test_step1)

In [99]:
scaler = TimeSeriesScalerMinMax()
X_train = scaler.fit_transform(X_train_s1)
X_test = scaler.transform(X_test_s1)

In [72]:
n_ts, ts_sz = X_train_s1.shape[:2]
n_classes = len(set(y_train1))

In [81]:
X_train_s1.shape

(173, 248, 40)

In [76]:
shapelet_sizes = grabocka_params_to_shapelet_size_dict(n_ts=n_ts,
                                                       ts_sz=ts_sz,
                                                       n_classes=n_classes,
                                                       l=0.1,
                                                       r=1)

In [98]:
shapelet_sizes

{24: 5}

In [96]:
X_train_s1.shape

(173, 248, 40)

In [97]:
y_train1

0         1
2102      5
3212      1
4263      6
5296      7
         ..
260433    9
261541    9
262655    9
263774    9
264886    9
Name: DeviationID ValueY, Length: 173, dtype: int32

In [100]:
shp_clf = LearningShapelets(n_shapelets_per_size=shapelet_sizes,
                            optimizer=tf.optimizers.Adam(.01),
                            batch_size=16,
                            weight_regularizer=.01,
                            max_iter=250,
                            random_state=42,
                            verbose=0)
shp_clf.fit(X_train, y_train1)

C:\Users\paulh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tslearn\shapelets\shapelets.py:354: FutureWarning: The default value for 'scale' is set to False in version 0.4 to ensure backward compatibility, but is likely to change in a future version.
  warnings.warn("The default value for 'scale' is set to False "


LearningShapelets(batch_size=16, max_iter=250, n_shapelets_per_size={24: 5},
                  optimizer=<keras.optimizers.adam.Adam object at 0x00000275052EB910>,
                  random_state=42, weight_regularizer=0.01)

In [89]:
y_test1

1052       4
6299       8
7334       9
8449       9
11006      3
          ..
246713    10
248812    10
251971    10
256163     9
259318     9
Name: DeviationID ValueY, Length: 78, dtype: int32

In [102]:
pred_labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [101]:
pred_labels = shp_clf.predict(X_test_s1)
print("Correct classification rate:", accuracy_score(y_test1, pred_labels))

Correct classification rate: 0.11538461538461539


In [10]:
X_train = []
for i in X_train1.Batch.unique():
    for j in X_train1["CuStepNo ValueY"].unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == j)].drop(["Batch"],axis=1))
        X_train.append(num)
X_test = []
for i in X_test1.Batch.unique():
    for j in X_test1["CuStepNo ValueY"].unique():
        num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)&(X_test1["CuStepNo ValueY"] == j)].drop(["Batch"],axis=1))
        X_test.append(num)

In [11]:
X_train4 = TimeSeriesResampler(sz=100).fit_transform(X_train)
X_test4 = TimeSeriesResampler(sz=100).fit_transform(X_test)

In [13]:
scaler = TimeSeriesScalerMinMax()
X_train = scaler.fit_transform(X_train4)
X_test = scaler.transform(X_test4)

In [15]:
n_ts, ts_sz = X_train.shape[:2]

In [20]:
n_classes = len(set(y_train1))

In [23]:
shapelet_sizes = grabocka_params_to_shapelet_size_dict(n_ts=n_ts,
                                                       ts_sz=ts_sz,
                                                       n_classes=n_classes,
                                                       l=0.1,
                                                       r=1)

In [24]:
shp_clf = LearningShapelets(n_shapelets_per_size=shapelet_sizes,
                            optimizer=tf.optimizers.Adam(.01),
                            batch_size=16,
                            weight_regularizer=.01,
                            max_iter=200,
                            random_state=42,
                            verbose=0)
shp_clf.fit(X_train, y_train1)

C:\Users\paulh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tslearn\shapelets\shapelets.py:354: FutureWarning: The default value for 'scale' is set to False in version 0.4 to ensure backward compatibility, but is likely to change in a future version.
  warnings.warn("The default value for 'scale' is set to False "


LearningShapelets(batch_size=16, max_iter=200, n_shapelets_per_size={10: 5},
                  optimizer=<keras.optimizers.adam.Adam object at 0x000002740C200B80>,
                  random_state=42, weight_regularizer=0.01)

In [25]:
pred_labels = shp_clf.predict(X_test)
print("Correct classification rate:", accuracy_score(y_test1, pred_labels))

Correct classification rate: 0.38
